# Étape 2 — Nettoyage et préparation des données

Cette étape vise à corriger les problèmes de qualité identifiés lors de l’audit
notamment :
- les types de variables incorrects.
- les valeurs manquantes.
- la cohérence minimale des données.

Les données nettoyées seront sauvegardées dans le dossier `data/processed/`
afin d’être utilisées lors des analyses et des jointures ultérieures.


In [1]:
# Import des libraries
import pandas as pd
import numpy as np


# Chargement des données brutes

In [2]:
customers = pd.read_csv("../data/raw/customers.csv")
order_lines = pd.read_csv("../data/raw/order_lines.csv")
products = pd.read_csv("../data/raw/products.csv")


# Rappel des structures

In [3]:
customers.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   customer_id  500 non-null    object
 1   age          500 non-null    object
 2   gender       490 non-null    object
 3   city         485 non-null    object
 4   segment      500 non-null    object
 5   signup_date  500 non-null    object
dtypes: object(6)
memory usage: 23.6+ KB


In [4]:
order_lines.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 18 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   order_id          2225 non-null   object 
 1   customer_id       2225 non-null   object 
 2   product_id        2225 non-null   object 
 3   order_date        2225 non-null   object 
 4   quantity          2225 non-null   int64  
 5   unit_price        2225 non-null   float64
 6   discount_pct      2225 non-null   float64
 7   gross_amount      2225 non-null   float64
 8   net_amount        2225 non-null   float64
 9   payment_method    2225 non-null   object 
 10  channel           2225 non-null   object 
 11  marketing_source  2225 non-null   object 
 12  delivery_days     2181 non-null   float64
 13  returned          2225 non-null   int64  
 14  review_score      2115 non-null   float64
 15  segment           2225 non-null   object 
 16  city              2155 non-null   object 


In [5]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   product_id    60 non-null     object 
 1   category      60 non-null     object 
 2   brand         60 non-null     object 
 3   product_name  60 non-null     object 
 4   unit_price    60 non-null     float64
dtypes: float64(1), object(4)
memory usage: 2.5+ KB


## Nettoyage du fichier customers

Les principales actions réalisées sont :
- conversion de la variable `age` en numérique,
- conversion de `signup_date` en date,
- traitement des valeurs manquantes pour `gender` et `city`.


In [6]:
# Types
customers["age"] = pd.to_numeric(customers["age"], errors="coerce")
customers["signup_date"] = pd.to_datetime(customers["signup_date"], errors="coerce")

# Valeurs manquantes
customers["gender"] = customers["gender"].fillna("Unknown")
customers["city"] = customers["city"].fillna("Unknown")

# Doublons
customers = customers.drop_duplicates(subset="customer_id")


In [7]:
customers

,customer_id,age,gender,city,segment,signup_date
0,C0001,28.0,M,Gonaïves,Entreprise,2025-09-26
1,C0002,16.0,M,Gonaïves,Professionnel,2024-01-19
2,C0003,NaN,M,Saint-Marc,Indépendant,2024-05-05
3,C0004,46.0,F,Port-au-Prince,Professionnel,2024-11-16
4,C0005,19.0,M,Cap-Haïtien,Professionnel,2024-11-21
...,...,...,...,...,...,...
495,C0496,35.0,F,Kinshasa,Entreprise,2024-12-09
496,C0497,35.0,Autre,Port-au-Prince,Professionnel,2025-08-30
497,C0498,16.0,M,Port-au-Prince,Entreprise,2025-01-13
498,C0499,16.0,M,Paris,Indépendant,2023-09-04


In [8]:
#customers.info()
customers.isna().sum()


customer_id    0
age            8
gender         0
city           0
segment        0
signup_date    0
dtype: int64

## Nettoyage du fichier order_lines

Les actions principales incluent :
- conversion de la variable `order_date` en format `datetime`
- conversion de la variable `discount_product` en numérique
- traitement des valeurs manquantes
- gestion des doublons
- cohérence des anomalies
- préparation des données pour les contrôles de cohérence ultérieurs.


**Taille avant nettoyage**

In [9]:
print("Taille initiale :", order_lines.shape)


Taille initiale : (2225, 18)


**Types**

In [10]:
# Order_date en datetime
order_lines["order_date"] = pd.to_datetime(order_lines["order_date"], errors="coerce")

In [11]:
# discount_product en numerique
order_lines["discount_pct"] = pd.to_numeric(order_lines["discount_pct"], errors="coerce")

# Valeurs manquantes

**Justification du choix de la méthode d’imputation (Coefficient d'asymétrie)**

Avant d’imputer les valeurs manquantes des variables numériques, il est nécessaire
d’examiner la forme de leur distribution.

Le coefficient d’asymétrie mesure le degré de dissymétrie d’une distribution :
- un coefficient d’asymétrie proche de 0 indique une distribution approximativement symétrique.
- un coefficient d’asymétrie positive indique une asymétrie à droite.
- un coefficient d’asymétrie négative indique une asymétrie à gauche.

Règle de décision :
- si la distribution est approximativement symétrique (|skewness| < 0.5), 
  nous allons imputé les valeurs manquantes par la moyenne.
- si la distribution est asymétrique (|skewness| ≥ 0.5), nous allons imputé 
  les valeurs manquantes par la médiane.

Pour la variable catégorielle (`city`), on remplace les valeurs maquantes
par la modalité `"Unknown"`.

**Calcul du coefficient d'asymétrie** 

Variables concernées 
- `delivery_days`
- `review_score`

In [12]:
# Calcul du coefficient d’asymétrie (skewness) pour les variables concernées

skew_delivery = order_lines["delivery_days"].skew()
skew_review = order_lines["review_score"].skew()

print("Coefficient d’asymétrie de delivery_days :", abs(skew_delivery))
print("Coefficient d’asymétrie de review_score :", abs(skew_review))


Coefficient d’asymétrie de delivery_days : 2.7982806670534495
Coefficient d’asymétrie de review_score : 0.3375988203615006


**Imputation**
- Les valeurs manquantes de la variable `delivery_days` seront imputées par la médiane
- Les valeurs manquantes de la variable `review_score` seront imputées par la moyenne
- Les valeurs manquantes de la variable `city` seront imputées par la modalité `"Unknown"`

In [13]:
# Imputation des valeurs manquantes

order_lines["delivery_days"] = order_lines["delivery_days"].fillna(order_lines["delivery_days"].median())
order_lines["review_score"] = order_lines["review_score"].fillna(order_lines["review_score"].mean())
order_lines["city"] = order_lines["city"].fillna("Unknown")


**Doublons**

In [14]:
order_lines = order_lines.drop_duplicates(subset=["order_id", "product_id"])

**Anomalies**

In [15]:
# Anomalies détectées avant nettoyage

print("Quantités <= 0 :", (order_lines["quantity"] <= 0).sum())
print("delivery_days > 30 :", (order_lines["delivery_days"] > 30).sum())

Quantités <= 0 : 6
delivery_days > 30 : 6


**Vérification de cohérences**

In [16]:
# Vérification de la Coherence des montants

order_lines["net_calc"] = (
    order_lines["gross_amount"] * (1 - order_lines["discount_pct"])
)

order_lines["amount_diff"] = order_lines["net_amount"] - order_lines["net_calc"]

print("Lignes incohérentes (|diff| > 0.01) :",
      (order_lines["amount_diff"].abs() > 0.01).sum())


Lignes incohérentes (|diff| > 0.01) : 0


**Nettoyage**

In [17]:

order_lines_clean = order_lines.copy()

order_lines_clean = order_lines_clean[order_lines_clean["quantity"] > 0]               # Quantités non valides
order_lines_clean = order_lines_clean[order_lines_clean["delivery_days"] <= 30]        # Délai de livraison excessif
order_lines_clean = order_lines_clean[order_lines_clean["amount_diff"].abs() <= 0.01]


**Taille après nettoyage**

In [18]:
print("Taille après nettoyage :", order_lines_clean.shape)
print("Lignes supprimées :", order_lines.shape[0] - order_lines_clean.shape[0])

Taille après nettoyage : (2188, 20)
Lignes supprimées : 12


**Aperçu**

In [19]:

order_lines_clean.head()


,order_id,customer_id,product_id,order_date,quantity,unit_price,discount_pct,gross_amount,net_amount,payment_method,channel,marketing_source,delivery_days,returned,review_score,segment,city,category,net_calc,amount_diff
0,O00001,C0201,P031,2024-06-16,4,165.54,0.18,662.16,542.97,Carte,App,Organique,1.0,0,5.0,Indépendant,Port-au-Prince,Cours,542.9712,-0.0012
1,O00002,C0141,P041,2024-06-10,5,46.48,0.14,232.40,199.86,Carte,Web,Réseaux sociaux,0.0,0,4.0,Étudiant,Abidjan,Livre,199.8640,-0.0040
2,O00003,C0025,P005,2024-08-29,2,1123.49,0.04,2246.98,2157.10,Virement,Web,Réseaux sociaux,3.0,0,4.0,Étudiant,Cap-Haïtien,Laptop,2157.1008,-0.0008
3,O00004,C0188,P042,2024-01-27,2,68.31,0.22,136.62,106.56,Carte,Boutique,Partenariat,0.0,0,4.0,Indépendant,Abidjan,Cloud,106.5636,-0.0036
4,O00005,C0023,P052,2025-08-06,3,1932.53,0.17,5797.59,4812.00,Carte,Web,Référencement payant,9.0,0,5.0,Professionnel,Cap-Haïtien,Laptop,4811.9997,0.0003


# Fichier Products
- Déjà propres
- Pas de valeurs manquantes
- Pas de doublons attendus

In [20]:
products = products.drop_duplicates(subset="product_id")

**Export des jeux de données vers le dossier data/processed**

In [21]:
# Sauvegarde du fichier customers nettoyé
customers.to_csv("../data/processed/customers_clean.csv", index=False)


In [22]:
# Sauvegarde du fichier order_lines nettoyé
order_lines_clean.to_csv("../data/processed/order_lines_clean.csv", index=False)

In [23]:
# Sauvegarde du fichier products 
products.to_csv("../data/processed/products_clean.csv", index=False)


## Règles de nettoyage appliquées

**Règle 1 — Correction des types de variables**  
Les variables représentant des dates (`signup_date`, `order_date`) ont été converties
au format datetime. Les variables numériques (`age`, `discount_pct`) ont été converties
en type numérique, avec transformation des valeurs non valides (ex. `"unknown"`) en
valeurs manquantes.

**Règle 2 — Traitement des valeurs manquantes**  
Les valeurs manquantes ont été traitées selon leur nature : imputation par la médiane
pour la variable numérique `delivery_days` et imputation par la moyenne pour la variable 
numérique `review_score` afin de limiter l’effet des valeurs extrêmes 
et remplacement par la modalité `"Unknown"` pour les variables
catégorielles (`gender`, `city`).

**Règle 3 — Gestion des doublons**  
Les doublons ont été détectés sur la base des clés métiers pertinentes
(`customer_id` pour les clients, `order_id` et `product_id` pour les ventes) et supprimés
afin d’éviter tout double comptage dans les analyses.

**Règle 4 — Détection et suppression des anomalies métiers**  
Les observations présentant des quantités nulles ou négatives (`quantity ≤ 0`) ainsi que
des délais de livraison excessifs (`delivery_days > 30`) ont été supprimées, ces valeurs
étant jugées non réalistes d’un point de vue métier.

**Règle 5 — Contrôle de cohérence des montants financiers**  
La cohérence entre le montant brut, la remise et le montant net a été vérifiée selon la
relation : `net_amount ≈ gross_amount × (1 − discount_pct)`. Les lignes pour lesquelles
l’écart dépassait une tolérance de 0,01 (liée aux arrondis) ont été exclues du jeu de données
final.
